## Literature Review

What:
TextRank uses an extractive approach and is an unsupervised graph-based text summarization technique. 

How: 
The basic idea implemented by a graph-based ranking model is that of voting or recommendation.
When one vertex links to another one, it is basically casting a vote for that vertex. The higher the number of votes cast for a vertex, the higher the importance of that vertex.



## Implementation

### Prerequisite

untuk kelancaran TextRank ini, diperlukan untuk :
1. men-download library stopwords Bahasa Indonesia: Python Sastrawi.
    Cara download library:
    1. Open terminal
    2. write on command line : "pip install Sastrawi"

### Libraries

In [1]:
import nltk
import math

import urllib
import os, os.path
#import modular regular expression
import re
import string
# import StopWordRemoverFactory class
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from nltk.cluster.util import cosine_distance
import numpy as np

from operator import itemgetter 

### Open txt file and Preprocessing

Preprocessing methods used:
1. decode to UTF-8
2. change all to lower case -> consistency purposes especially during stopwords removal process
3. remove all numbers -> avoid redundancy
4. remove all punctuation -> avoid redundancy
5. stopwords (using Python Sastrawi) -> avoid redundancy
6. Stemmer (using Python Sastrawi) -> easy filter

In [ ]:
url = "https://raw.githubusercontent.com/jordhy97/final_project/master/data/reviews/reviews.txt"
file = urllib.request.urlopen(url)

stopword = StopWordRemoverFactory().create_stop_word_remover()
stemmer = StemmerFactory().create_stemmer()

aggregated_lines = ""

for line in file:
    decoded_line = line.decode("utf-8")
    lower_decoded_line = decoded_line.lower()
    num_rem_decoded_line = re.sub(r"\d+", "", lower_decoded_line)
    punc_rem_decoded_line = num_rem_decoded_line.translate(str.maketrans("","",string.punctuation))
    sw_decoded_line = stopword.remove(punc_rem_decoded_line)
    katadasar_line = stemmer.stem(sw_decoded_line)
    print(katadasar_line)
    aggregated_lines = aggregated_lines + "[" +aggregated_lines + "], " +"\n"

kotor debu tdk henti bersin awal masuk kamar kesan kamar kumuh wifi rusak lgsg check out
oke cuma air wastafel warna keruh mohon baik
kamar semut kamar mandi masalah buang air mampetsehingga air luber luar kamar
kamar mandi bau air bau
kamar bersih nyaman perlu perhati dan trus kroscheck trutama slot card pintu kamar batrai nya habis apa msih full kasi ada tamu lelah habis jalan jauh tunggu luar karena slotcard pintu kamar di perbaikin ulang tks smoga manfaat
tak sesuai espektasi kamar sempit pintu kamar mandi rusak cek in tunggu terlalu lama malam suara berisik dr atap kamar nyaman
booking traveloka room sedia kembali uang nya lamaaa
pesan kamar double bad dapat kamar twins bad
buruk kasur bekas sperma seprai jg air bau karat kasur per jg rusak sabun habis pasword wifi ga tau
baik puas masih kurang waktu boking airy rooms pilih bad tp saya cek in kasih bad mohon di tingkat terimah kasih airy rooma
kamar mandi kurang bersih kurang lengkap faslitasnya kamar mandi yang lumayan
lokasi sus

### Modelling

we first build a graph associated with the text, where the graph vertices are representative for the units to be ranked. The goal is to rank entire sentences, therefore, a vertex is added to the graph for each sentence in the text.

Here are some conditions a good similarity measure has to obey:

1. 0 <= similarity(A, B) <= 1
2. similarity(A, A) = 1
3. similarity(A, B) =/= 1 if A =/= B

Note: A and B are different sentences.

A widely used measure in Natural Language Processing is the Cosine Similarity. The Cosine Similarity computes the cosine of the angle between 2 vectors. If the vectors are identical, the cosine is 1.0. If the vectors are orthogonal, the cosine is 0.0. This means the cosine similarity is a measure we can use. NLTK implements cosine_distance, which is 1 - cosine_similarity. The concept of distance is opposite to similarity. Two identical vectors are located at 0 distance and are 100% similar.

#### Sentence Similarity using Cosine Similarity

In [58]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

#### Building the similarity Matrix

In [ ]:
def build_similarity_matrix(aggregated_lines, stopwords=None):
    # Create an empty similarity matrix
    S = np.zeros((len(aggregated_lines), len(aggregated_lines)))
 
    for idx1 in range(len(aggregated_lines)):
        for idx2 in range(len(aggregated_lines)):
            if idx1 == idx2:
                continue
 
            S[idx1][idx2] = sentence_similarity(aggregated_file[idx1], aggregated_file[idx2], stop_words)
 
    # normalize the matrix row-wise
    for idx in range(len(S)):
        S[idx] /= S[idx].sum()
 
    return S
 
print("Len of aggregated lines: " + len(aggregated_lines))

stop_words = ""
S = build_similarity_matrix(aggregated_lines, stop_words)    
print(S)

#### Compute the Summary

In [ ]:
sentence_ranks = pagerank(S)
 
print(sentence_ranks)
 
# Get the sentences ordered by rank
ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
print(ranked_sentence_indexes)
 
# Suppose we want the 5 most import sentences
SUMMARY_SIZE = 5
SELECTED_SENTENCES = sorted(ranked_sentence_indexes[:SUMMARY_SIZE])
print(SELECTED_SENTENCES)
 
# Fetch the most important sentences
summary = itemgetter(*SELECTED_SENTENCES)(aggregated_lines)
 
# Print the actual summary
for aggregated_line in summary:
    print(' '.join(aggregated_line))

#### Driver Code

In [ ]:
def textrank(aggregated_lines, top_n=5, stopwords=None):
    """
    aggregated_lines = a list of airy review sentences 
    top_n = how may sentences the summary should contain
    stopwords = a list of stopwords
    """
    S = build_similarity_matrix(aggregated_lines, stop_words) 
    sentence_ranks = pagerank(S)
 
    # Sort the sentence ranks
    ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
    selected_sentences = sorted(ranked_sentence_indexes[:top_n])
    summary = itemgetter(*selected_sentences)(aggregated_lines)
    return summary
 
for idx, aggregated_line in enumerate(textrank(aggregated_lines, stopwords=stopwords.words('english'))):
    print("%s. %s" % ((idx + 1), ' '.join(aggregated_line)))

### Evaluating

for the purpose of this project, we will be using ROUGE and BLEU metrics

#### ROUGE Metrics

#### BLEU Metrics

## Main Reference

- Preprocessing techniques ideas are inspired by the links below: 
    1. https://www.academia.edu/35015140/Preprocessing_Techniques_for_Text_Mining
    2. https://medium.com/@ksnugroho/dasar-text-preprocessing-dengan-python-a4fa52608ffe

- References for library for Indonesian Stopwords: 
    1. https://medium.com/@ksnugroho/dasar-text-preprocessing-dengan-python-a4fa52608ffe
    2. https://devtrik.com/python/stopword-removal-bahasa-indonesia-python-sastrawi/
    
- Code for the modelling of the project is mainly referred from this website: 
    1. https://nlpforhackers.io/textrank-text-summarization/ 
    2. I do not take ownership of the code presented above

## Additional Reference

Below are links I have referred to throughout the making of the project:
- TextRank algorithms: 
    1. https://iq.opengenus.org/textrank-for-text-summarization/
    2. https://github.com/summanlp/textrank
    3. https://github.com/davidadamojr/TextRank
    4. https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf
    5. https://medium.com/@shivangisareen/text-summarisation-with-gensim-textrank-46bbb3401289
    6. https://towardsdatascience.com/understand-text-summarization-and-create-your-own-summarizer-in-python-b26a9f09fc70
- Text Summarization algorithm using word frequency: 
    1. https://becominghuman.ai/text-summarization-in-5-steps-using-nltk-65b21e352b65